# Matrix Product States

In [2]:
import numpy as np

## TT-SVD

In [3]:
N=10 # number of sites/spins
d=2 # physical dimension

eps=0 # SVD truncation error

tens = np.random.rand(*[d] * N) # high-dimensional tensor
A=[] # list for storing MPS tensors
tens.shape

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)

In [4]:
# Reshape (step 1)
tmp = tens.reshape(tens.shape[0], -1) # (temporary tensor)
tmp.shape

(2, 512)

In [5]:
# SVD + Truncate (step 2)
U, s, Vt = np.linalg.svd(tmp)

# Truncate singular values such that truncation error is less than or equal to eps
where_truncation_error_is_lower_than_eps = np.where(np.cumsum(s[::-1]**2) <= eps**2)[0]
num_sv_to_discard = 0 if len(where_truncation_error_is_lower_than_eps) == 0 else int(1 + where_truncation_error_is_lower_than_eps[-1])
r = max(1, len(s) - num_sv_to_discard) # new rank

In [6]:
# Reshape and truncate U matrix, store as first MPS site
A.append(U[:,:r].reshape(1, d, r))
A[0].shape

(1, 2, 2)

In [7]:
# Contract s and Vt (step 3)
tmp = np.diagflat(s[:r]) @ Vt[:r,:]
tmp.shape

(2, 512)

In [8]:
# Reshape (step 4)
tmp = tmp.reshape(r * tens.shape[1], -1)
tmp.shape
# Repeat steps 2-4, N-1 times

(4, 256)

In [9]:
from copy import copy
def tt_svd(tens: np.ndarray, eps: float = 10**-6, rank = 10**12) -> list:
    """
    Compress a tensor to a MPS/TT using the TT-SVD algorithm.

    Args:
        tens: The input tensor
        eps: Truncation error for each SVD
    Return:
        An MPS/TT as a list of order-3 tensors (dummy bonds are added to boundary tensors)
    """
    dims = tens.shape
    N = len(dims)
    tmp = copy(tens)
    A = []
    r_prev = 1
    for i in range(N-1):
        # Reshape (step 4)
        tmp = tmp.reshape(r_prev * dims[i], -1)
        
        # SVD + Truncate (step 2)
        U, s, Vt = np.linalg.svd(tmp)
        # Truncate singular values such that truncation error is less than or equal to eps
        where_truncation_error_is_lower_than_eps = np.where(np.cumsum(s[::-1]**2) <= eps**2)[0]
        num_sv_to_discard = 0 if len(where_truncation_error_is_lower_than_eps) == 0 else int(1 + where_truncation_error_is_lower_than_eps[-1])
        r = min(rank, max(1, len(s) - num_sv_to_discard)) # new rank
        
        # Reshape and truncate U matrix, store in return list
        A.append(U[:,:r].reshape(r_prev, dims[i], r))
        
        # Contract s and Vt (step 3)
        tmp = np.diagflat(s[:r]) @ Vt[:r,:]
        r_prev = r
    A.append(tmp.reshape(r_prev, dims[-1], 1))
    return A

In [10]:
eps=10**-1
mps = tt_svd(tens, eps=eps)

In [11]:
# Show bond dimensions/tt-ranks
def bdims(mps: list):
    return [site.shape[0] for site in mps] + [mps[-1].shape[-1]]

In [12]:
bdims(mps)

[1, 2, 4, 8, 16, 31, 16, 8, 4, 2, 1]

## Reconstructing full tensor 

In [13]:
def restore_full(mps: list) -> np.ndarray:
    """
    Restore full tensor from an MPS/TT

    Args:
        mps: List of order-3 tensors representing an MPS/TT

    Return:
        The full tensor
    """
    tmp = mps[0]
    dims = [site.shape[1] for site in mps]
    for site in mps[1:]:
        tmp = np.einsum('iuj,jvk->iuvk', tmp, site)
        tmp = tmp.reshape(tmp.shape[0], tmp.shape[1] * tmp.shape[2], tmp.shape[3])
    return tmp.reshape(dims)

In [14]:
# The TT/MPS approximation error
np.linalg.norm(tens - restore_full(mps)) 

0.04256018024345947

In [15]:
# Theoreterical upper bound for the TT/MPS approximation error (if each SVD truncation error is <= eps)
eps * np.sqrt(N)

0.316227766016838

In [16]:
# Number of parameters in MPS
sum(np.count_nonzero(site) for site in mps)

2664

In [17]:
# Number of parameters in orginal tensor
np.count_nonzero(tens)

1024

## Tensors with low TT-rank

The above random tensor doesn't have a low TT-rank and is not approximated well by a MPS/TT with low rank. Below is an example of a tensor which does have a low TT-rank (rank-2): a sinusoidal signal reshaped into a tensor. An even lower TT-rank tensor (rank-1) is that given by reshaping the exponential function.

In [60]:
N=10
d=2

w=1
phi=0.5
g=0.1
def x(t):
    return np.sin(w*t + phi)
    # return np.exp(-g*t)

tens = np.fromiter((x(t) for t in range(d**N)), dtype=np.float64).reshape([d] * N)
mps = tt_svd(tens, eps=10**-10)

In [61]:
bdims(mps)

[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]

In [62]:
np.linalg.norm(tens - restore_full(mps)) 

8.481668898998306e-14

In [63]:
sum(np.count_nonzero(site) for site in mps)

72

In [64]:
np.count_nonzero(tens)

1024

## Getting tensor elements in the MPS/TT format

In [49]:
from functools import reduce
def get_index(mps: list, inds):
    return reduce(np.matmul, (site[:, ind, :] for site, ind in zip(mps, inds)))

In [68]:
get_index(mps, [0, 1, 1, 0, 0, 1, 0, 1, 1, 1])

array([[-0.78768666]])

In [69]:
tens[0, 1, 1, 0, 0, 1, 0, 1, 1, 1]

-0.7876866561270567